# Tutorial 2: Creating or Opening a SIR 3S Model

This Tutorial demonstrates how to create new SIR 3S Models or open already existing ones.

# Toolkit Release

In [1]:
#pip install sir3stoolkit

# Imports

## DLL References

The Toolkit requires the Sir3S_Toolkit.dll included in SIR 3S installations (from Quebec and Up). Furthermore the Sir3S_Repository.Interfaces.dll is required to input native SIR 3S datatypes into some Toolkit functions.

In [2]:
import clr as clr

In [3]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S Entwicklung\SirGraf-90-15-00-10_Quebec.beta_x64"

In [4]:
clr.AddReference(SIR3S_SIRGRAF_DIR+r"\Sir3S_Repository.Interfaces")
import Sir3S_Repository.Interfaces as Interfaces

In [5]:
clr.AddReference(SIR3S_SIRGRAF_DIR+r"\Sir3S_Toolkit")
import Sir3S_Toolkit

## PythonWrapperToolkit

In [6]:
import sir3stoolkit

The core of sir3stoolkit is a Python Wrapper around C#, that can be used to read, write, etc. to a SIR 3S Model. In the future pure python subpackages may be added.

In [7]:
from sir3stoolkit.core import wrapper

In [8]:
sir3stoolkit

<module 'sir3stoolkit' from 'c:\\Users\\jablonski\\AppData\\Local\\anaconda3\\Lib\\site-packages\\sir3stoolkit\\__init__.py'>

In [9]:
sir3stoolkit

<module 'sir3stoolkit' from 'c:\\Users\\jablonski\\AppData\\Local\\anaconda3\\Lib\\site-packages\\sir3stoolkit\\__init__.py'>

An instance of the wrapper package has to be created with reference to a SIR Graf installation.

In [10]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Addtiontional

In [11]:
import os

In [12]:
import requests

# Initialization

Toolkit contains two classes: SIR3S_Model(model and data) and SIR3S_View(depiction in SIR Graf). All Toolkit functionality is accessed via the methods of these classes.

In [13]:
s3s = wrapper.SIR3S_Model()

Initialization complete


In [14]:
s3s_view = wrapper.SIR3S_View()

Initialization complete


# Get Data

A local folder next to the notebook will be created. That contains the model that will be opened later.

In [15]:
def download_directory(owner, repo, directory, local_dir, branch="master", headers=None):
    """
    Recursively download a directory from a GitHub repository.
    """
    if headers is None:
        headers = {}

    # GitHub API URL for the directory
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{directory}?ref={branch}"

    # Get directory contents
    response = requests.get(api_url, headers=headers)
    response.raise_for_status()
    items = response.json()

    # Create local directory if it doesn't exist
    os.makedirs(local_dir, exist_ok=True)

    for item in items:
        item_path = item["path"]
        item_type = item["type"]
        item_name = item["name"]
        local_path = os.path.join(local_dir, item_name)

        if item_type == "file":
            file_url = item["download_url"]
            file_data = requests.get(file_url)
            with open(local_path, "wb") as f:
                f.write(file_data.content)
        elif item_type == "dir":
            # Recursively download subdirectory
            download_directory(owner, repo, item_path, local_path, branch, headers)

In [16]:
# Configuration
owner = "mj3s"
repo = "SIR-3S_Example_Data"
directory = "sir3stoolkit/tutorials/Toolkit_Tutorial_2"
branch = "master"
local_dir = "Toolkit_Tutorial_2"
headers = {}  # Add token here if needed

# Start downloading
download_directory(owner, repo, directory, local_dir, branch, headers)

# Create New Model

We are using the NewModel() function of SIR3S_Model() to create a new db3 file.

In [17]:
dbFilePath=r"Tutorial_2_NewModel.db3"

In [18]:
s3s.NewModel(dbName=dbFilePath,
                           providerType=Interfaces.SirDBProviderType.SQLite, 
                           namedInstance="", 
                           netType=Interfaces.NetworkType.Undefined,
                           modelDescription="Tutorial New Model",
                           userID="", 
                           password="")

New model is created with the model identifier: M-1-0-1


Now the Model has been created and is opened. All Toolkit commands now apply to this model until another one is opened. If there already existed a model file with this path and name, a new model with the same name but incremented model identifier is created.

In [19]:
print(s3s.GetNetworkType()) # to check that the model is responsive

Undefined


# Open Model

In [20]:
dbFilePath=r"Tutorial_2_OpenModel.db3"

In [21]:
s3s.OpenModel(dbName=dbFilePath, 
                            providerType=Interfaces.SirDBProviderType.SQLite, 
                            Mid="M-1-0-1", 
                            saveCurrentlyOpenModel=False, 
                            namedInstance="", 
                            userID="", 
                            password="")

Error while opening the model, Could not open Database using Connection String 'Provider=System.Data.SQLite;Data Source="Tutorial_2_OpenModel.db3";Version=3;'


Now the Model has been opened the previous one was close without saving it. All Toolkit commands now apply to this model until another one is opened.

In [22]:
print(s3s.GetNetworkType()) # to check that the correct model is responsive

Undefined
